In [1]:
import os
import warnings
from dotenv import load_dotenv
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
SERPER_API_KEY = os.getenv("SERPER_API_KEY")
warnings.filterwarnings('ignore')

os.environ["SERPER_API_KEY"]=SERPER_API_KEY
os.environ['OPENAI_API_KEY']=OPENAI_API_KEY

In [2]:
from openai import OpenAI

client = OpenAI()
client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[{
        "role": "user",
        "content": "2002년 월드컵 4강 국가 알려줘"
    }]
)

ChatCompletion(id='chatcmpl-AkUXc9z8DZSf9Uvj5fVZ5khSihQdW', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='2002년 월드컵 4강에는 독일, 브라질, 터키, 한국이 진출했습니다. 이 중에서 브라질이 우승을 차지했습니다. 이웃님한테 도움이 되었길 바랍니다.', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None))], created=1735643852, model='gpt-3.5-turbo-0125', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=90, prompt_tokens=31, total_tokens=121, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))

In [3]:
from langchain_openai import ChatOpenAI
chat=ChatOpenAI(
    model_name='gpt-4o-mini'
)
chat.invoke("안녕~ 너를 소개해 줄래?")

AIMessage(content='안녕하세요! 저는 AI 언어 모델로, 다양한 질문에 답하거나 정보를 제공하는 역할을 합니다. 여러분의 궁금증을 해결하고, 대화를 나누며 도움을 드릴 수 있어요. 무엇을 도와드릴까요?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 53, 'prompt_tokens': 17, 'total_tokens': 70, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_d02d531b47', 'finish_reason': 'stop', 'logprobs': None}, id='run-c4ee3604-e042-4281-af69-217af497a940-0', usage_metadata={'input_tokens': 17, 'output_tokens': 53, 'total_tokens': 70, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [4]:
from langchain.prompts import PromptTemplate

prompt = (
    PromptTemplate.from_template(
        """
        너는 요리사야. 내가 가진 재료들을 갖고 만들 수 있는 요리를 {개수}개 추천하고,
        그 요리의 레시피를 제시해 줘. 내가 가진 재료는 아래와 같아.
        <재료>
        {재료}
        """
    )
)
prompt

PromptTemplate(input_variables=['개수', '재료'], input_types={}, partial_variables={}, template='\n        너는 요리사야. 내가 가진 재료들을 갖고 만들 수 있는 요리를 {개수}개 추천하고,\n        그 요리의 레시피를 제시해 줘. 내가 가진 재료는 아래와 같아.\n        <재료>\n        {재료}\n        ')

In [5]:
prompt.invoke({"개수": 2, "재료": "사과, 잼"})

StringPromptValue(text='\n        너는 요리사야. 내가 가진 재료들을 갖고 만들 수 있는 요리를 2개 추천하고,\n        그 요리의 레시피를 제시해 줘. 내가 가진 재료는 아래와 같아.\n        <재료>\n        사과, 잼\n        ')

In [7]:
from langchain_core.prompts import ChatPromptTemplate

chat_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful AI bot. Your name is {name}"),
        ("human", "Hello, how atr you doing?"),
        ("ai", "I'm doing well, thinks!"),
        ("human", "{user_input}")
    ]
)

message = chat_template.format_messages(name="Bob", user_input="What is your name?")
print(message)

[SystemMessage(content='You are a helpful AI bot. Your name is Bob', additional_kwargs={}, response_metadata={}), HumanMessage(content='Hello, how atr you doing?', additional_kwargs={}, response_metadata={}), AIMessage(content="I'm doing well, thinks!", additional_kwargs={}, response_metadata={}), HumanMessage(content='What is your name?', additional_kwargs={}, response_metadata={})]


In [8]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_template("tell me a short joke about {topic}")

model = ChatOpenAI(model="gpt-4o-mini")

chain=prompt|model|StrOutputParser()
chain.invoke({"topic": "ice cream"})

'Why did the ice cream cone break up with the sundae?\n\nBecause it found someone cooler!'

In [9]:
model = ChatOpenAI(model="gpt-4o")
prompt=ChatPromptTemplate.from_template("tell me a short joke about {topic}")
chain = prompt|model

for s in chain.stream({"topic": "bears"}):
    print(s.content, end="", flush=True)

Why don't bears wear shoes?  
Because they prefer bear feet!

In [11]:
from langchain_openai import ChatOpenAI
from langchain.prompts import HumanMessagePromptTemplate
from langchain_core.messages import SystemMessage
from langchain_core.prompts import ChatPromptTemplate

llm=ChatOpenAI(model="gpt-4o-mini", temperature=0)

chat_template=ChatPromptTemplate.from_messages(
    [
        SystemMessage(
            content=("너는 영화 전문가 AI야. 사용자가 원하는 장르의 영화를 리스트 형태로 추천해줘. ex. Query: SF 영화 3개를 추천해 줘/ 답변: ['인터스텔라', '스페이스오디세이','혹성탈출']")
        ),
        HumanMessagePromptTemplate.from_template("{text}")
    ]
)
chain=chat_template|llm
chain.invoke("액션")

AIMessage(content="['다크 나이트', '존 윅', '매드 맥스: 분노의 도로']", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 73, 'total_tokens': 98, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0aa8d3e20b', 'finish_reason': 'stop', 'logprobs': None}, id='run-87b7e8af-f085-4600-91fb-dc9dba27c864-0', usage_metadata={'input_tokens': 73, 'output_tokens': 25, 'total_tokens': 98, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [12]:
from langchain.output_parsers import CommaSeparatedListOutputParser
from langchain.prompts import PromptTemplate

output_parser = CommaSeparatedListOutputParser()
format_instructions = output_parser.get_format_instructions()

prompt=PromptTemplate(
    template="List {subject}. answer in Korean \n{format_instructions}",
    input_variables=["subject"],
    partial_variables={"format_instructions": format_instructions}
)

model = ChatOpenAI(model="gpt-4o-mini")

chain=prompt|model|output_parser
chain.invoke({"subject": "공포 영화"})

['사이코', '엑소시스트', '할로윈', '더 링', '컨저링', '귀신이 산다', '피라냐', '노웨어', '저주', '13일의 금요일']

In [13]:
from typing import List
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_openai import ChatOpenAI

class Country(BaseModel):
    continent: str = Field(description="사용자가 물어본 나라가 속한 대륙")
    population: str = Field(description="사용자가 물어본 나라의 인구(int 형식)")

parser=JsonOutputParser(pydantic_object=Country)
prompt=PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()}
)

chain=prompt|model|parser

country_query = "아르헨티나는 어떤 나라야?"
chain.invoke({"query": country_query})

{'continent': 'South America', 'population': '45195777'}